# Modelos de Regresión Logística

## 1. Introducción a la Regresión Logística

Se trata de un **algoritmo estadístico** cuyo objetivo es aproximar una **función matemática acotada** en el intervalo [0, 1]. Así permite la estimación de las **probabilidades de pertenencia** de cada muestra a las distintas clases. Como la detección de sexismo es un problema de clasificación binario, se establece el umbral en 0.5 con el que asignar una clase a cada instancia aplicando las siguientes casuísticas:

* Si la probabilidad es **menor que 0.5** entonces la clase es **no sexista** (categoría 0). 
* Si la probabilidad es **mayor que 0.5** entonces la clase es **sexista** (categoría 1).

### 1.1. Condiciones de uso

* **Mínima correlación** entre los predictores o variables independientes.
* La existencia de una **relación lineal** entre los predictores y la probabilidad de cada variable dependiente.
* Mínima existencia de **outliers**.
* Conjunto de datos razonáblemente amplio.

### 1.2. Casos de uso

1. Construir **modelos lineales** para solventar problemas de clasificación cuyos datos son fácilmente separables.

2. Comprobar de la existencia de **relaciones lineales** entre los predictores y la variable dependiente en función de los valores de los coeficientes del hiperplano.

   - Si es 0 significa que **no existe una relación lineal** entre un predictor y la variable dependiente por lo que dicho predictor no ayuda en su predicción.

   - Si es mayor que 0 supone un crecimiento simultáneo del predictor al aumentar la variable dependiente, mientras que si es menor que 0 simboliza el incremento del predictor cuando disminuye la variable dependiente. Aunque en ambos casos el **predictor puede ser útil para predecir** la clase de una muestra.

## 2. Estructura del notebook

1. Introducción a la Regresión Logística
2. Estructura del notebook
3. Instalación y carga de librerías
4. Lectura y carga de datos
5. Técnicas de codificación de textos

## 3. Instalación y carga de librerías

Este apartado tiene como único propósito cargar las librerías y dependencias necesarias para la ejecución de este notebook, así como las funciones propiamente desarrolladas. Previo a ello deberán ser instaladas bien ejecutando el script *setup.sh* mediante el comando `bash setup.sh` con permisos de ejecución en distribuciones Linux, o bien ejecutando el compando `pip install -r requirements.txt`.

In [1]:
import sys
sys.path.append('../scripts')

# Import data read and compute functions
from data import read_training_dataset, read_testing_dataset

# Import text preprocess functions
from processing import *

# Import LR models and validation metrics
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

[nltk_data] Downloading package stopwords to /home/lidia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 4. Lectura y carga de datos

En esta sección se pretende **cargar los datasets de entrenamiento y validación** procedentes de los correspondientes ficheros situados en la carpeta *data*. Al tener un **formato TSV** se deben leer como tablas aunque posteriormente se trabaje con ellos en formato *dataframe*. 

Tal y como se puede comprobar en los siguientes resultados las dimensiones de sendos conjuntos de datos se detallan a continuación:

* Conjunto de entrenamiento: **6.977 muestras**.
* Conjunto de validación: **4.368 muestras**.

In [2]:
# Read EXIST datasets
training_df = read_training_dataset()
testing_df = read_testing_dataset()

# Show the dimensions of the datasets
print("Training dataset dimensions:", training_df.shape)
print("Testing dataset dimensions:", testing_df.shape)

Training dataset dimensions: (6977, 7)
Testing dataset dimensions: (4368, 7)


## 5. Técnicas de codificación de textos

### 5.1. Bolsa de palabras

Es una técnica orientada a la **extracción de características** basado en **texto** que convierte cada documento en un vector de longitud fija a partir del cálculo de la frecuencia de sus términos. Si bien se trata de un procedimiento altamente sencillo y rápido de aplicar, **no** respeta el **orden de los términos** en los documentos y tampoco se considera el **contexto** de los mismos como consecuencia.


## 6. Experimentos y modelos

A continuación se detallan los experimentos más relevantes de entre las distintas pruebas realizadas mediante diversas combinaciones de técnicas de procesamiento de textos, codificación de documentos y algoritmos clásicos de Aprendizaje Automático.

### 6.1. Primer experimento

- Procesamiento de los conjuntos de entrenamiento y validación, incluyendo las siguientes técnicas:

  - Elimina caracteres especiales, no alfabéticos y signos de puntuación.
  - Elimina hashtags y menciones de usuarios.
  - Elimina *stopwords* de los documentos en inglés y español.
  - Convierte todos los caracteres en minúsculas.

- Codificación de la variable dependiente *task1* a etiquetas numéricas en los conjuntos de entrenamiento y validación, estableciendo la asignación *non-sexist*~0 y *sexist*~1.

- Generación de una bolsa de palabras para entrenamiento y otra para validación como codificación de documentos en valores numéricos.

- Entrenamiento, hiperparametrización y validación con Regresión Logística para la construcción de varios modelos, siendo la siguiente configuración la que mejor resultados de validación ha conseguido:

In [3]:
# Text processing in training and testing datasets
training_df = text_processing_pipeline(training_df, "text")
testing_df = text_processing_pipeline(testing_df, "text")

# Encode the class labels to numeric values for training and testing datasets providing the desirable assignation
encoding = {"non-sexist": 0, "sexist": 1}
encoded_training_labels = encode_to_numeric_labels(training_df, "task1", encoding)
encoded_testing_labels = encode_to_numeric_labels(testing_df, "task1", encoding)

# Train a CountVectorizer object to encode texts as bag of words for training and testing datasets
bag_words = to_bag_of_words(list(training_df["cleaned_text"].values), 
                            list(testing_df["cleaned_text"].values))

# Create and configure a LR model
lr = LogisticRegression(random_state=1, solver='lbfgs', multi_class='ovr')

# Train the LR model
lr.fit(bag_words["training"], encoded_training_labels)

# Predict the testing labels with the trained LR model
predictions = lr.predict(bag_words["testing"])

# LR model validation
print("Accuracy %.3f" %metrics.accuracy_score(encoded_testing_labels, predictions))

Accuracy 0.689
